In [51]:
from scipy.io import loadmat
import numpy as np
import scipy
from sklearn.neighbors import NearestNeighbors
face_data = loadmat("FaceData.mat");
train_data = face_data["trainData"];
T = train_data
N,d = T.shape # number of samples,dimension of sample
c = N//2 #number of classes (people)

In [52]:
def normalize(T):
    global mean_pic
    T = T.astype(float)
    for index,sample in enumerate(T):
        sample = np.true_divide(sample,sample.std())
        sample = np.subtract(sample,sample.mean(0))
        assert round(sample.mean(0),2) == 0 #rounding errors may occur
        assert round(sample.std(0),2) == 1
        sample -= mean_pic
        T[index] = sample
    if mean_pic is 0:
        mean_pic = T.mean(0)
        return normalize(T)
    return T

In [58]:
mean_pic = 0
T = train_data
T = normalize(T)
A = np.column_stack(T)

In [59]:
S_prime = A.T @ A

In [60]:
def k_best_eig_values_vectors(M,k,B=None):
    a = scipy.linalg.eig(M,b=B)
    eig_value_vector_pairs = [(value,vector) for value,vector in zip(a[0],a[1])]
    best_eig_value_vectors = sorted(eig_value_vector_pairs,key=lambda x:x[0],reverse=True)[:k]
    best_eig_values = np.array([pair[0] for pair in best_eig_value_vectors])
    best_eig_vectors = np.array([pair[1] for pair in best_eig_value_vectors])
    return best_eig_values,best_eig_vectors

In [61]:
best_eig_values,best_eig_vectors = k_best_eig_values_vectors(S_prime,N-c)
V = np.column_stack(best_eig_vectors)
Z = np.diag(best_eig_values)
W_pca = A @ V @ Z

In [64]:
proj_T = np.array(T) @ W_pca
mean_pic = proj_T.mean(0)
# proj_T = normalize(proj_T)
face_pairs = np.array([(proj_T[i],proj_T[i+1]) for i,t in enumerate(proj_T[:-1]) if i%2 == 0])
mu = np.array([np.mean(pair,0) for pair in face_pairs])

In [69]:
def S_i(i):
    return sum(np.array([(face - mu[i]) @ np.array([(face - mu[i])]).T for face in face_pairs[i]]))
S_w = sum(np.array([S_i(i) for i in range(c)]))
S_b = sum(np.array([2*np.array([(mu_i - mean_pic)]).T@np.array([(mu_i - mean_pic)]) for mu_i in mu]))

In [64]:
best_eig_values,best_eig_vectors = k_best_eig_values_vectors(S_w,c-1,S_b)

In [65]:
best_eig_vectors

array([[ 0.31023919+0.j,  0.49329351+0.j,  0.23669429+0.j, ...,
         0.29289258+0.j,  0.20502109+0.j, -0.33253974+0.j],
       [ 0.00348234+0.j,  0.00197569+0.j,  0.01169005+0.j, ...,
         0.01628747+0.j, -0.00928397+0.j,  0.00154044+0.j],
       [-0.00397104+0.j,  0.00261942+0.j, -0.01204196+0.j, ...,
        -0.00448764+0.j,  0.00514064+0.j, -0.00560062+0.j],
       ..., 
       [-0.10086469+0.j, -0.12335900+0.j,  0.35022460+0.j, ...,
         0.37797026+0.j, -0.43023082+0.j, -0.56676843+0.j],
       [-0.31276321+0.j,  0.03527419+0.j, -0.15961962+0.j, ...,
        -0.28117501+0.j,  0.01926593+0.j,  0.02966516+0.j],
       [-0.28351759+0.j, -0.13247946+0.j,  0.16284675+0.j, ...,
        -0.13027443+0.j,  0.03575380+0.j,  0.19321467+0.j]])

In [71]:
W_mda = np.column_stack(best_eig_vectors)

In [72]:
W_opt = W_mda @ W_pca.T

In [74]:
train_data = face_data["trainData"]
test_data = face_data["testData"]
face_pairs = np.array([(train_data[i],train_data[i+1]) for i,t in enumerate(train_data[:-1]) if i%2 == 0])
models = np.array([np.mean(pair,0) for pair in face_pairs])
proj_models = np.array([W_opt @ model for model in models])

In [76]:
#classification
mean_pic = 0
normalize(test_data)
proj_test = np.array([W_opt @ test for test in test_data])
proj_test.shape

(50, 100)

In [77]:
nbrs = NearestNeighbors(n_neighbors=1).fit(proj_models)
distances , indices = nbrs.kneighbors(proj_test)

d:\bash\home\tadaboody\homework-collector\machine learning\virt_env\lib\site-packages\numpy\core\numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
d:\bash\home\tadaboody\homework-collector\machine learning\virt_env\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


In [78]:
correct = [index for index,img in enumerate(proj_test) if index == indices[index]]

In [79]:
suc = len(correct)/len(proj_test)

In [80]:
suc

0.52